In [85]:
import boto3
import pandas as pd
from io import StringIO

## Credentials

In [86]:
AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
AWS_REGION = "ap-south-1"
SCHEMA_NAME = "covid_19"
S3_STAGING_DIR = "s3://test-bucket-1306/output"
S3_BUCKET_NAME = "test-bucket-1306"
S3_OUTPUT_DIR = "output"

## boto3 Athena client

In [87]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

# s3_client = boto3.client(
#     "s3",
#     aws_access_key_id=AWS_ACCESS_KEY,
#     aws_secret_access_key=AWS_SECRET_KEY,
#     region_name=AWS_REGION
# )

## Function to convert tables in Glue Catalog to Pandas Dataframe

In [95]:
Dict = {}

def download_and_load_query_results(
    client: boto3.client, query_response: Dict
)-> pd.DataFrame:
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response['QueryExecutionId']
            )
            break
        except Exception as err:
            if 'not yet finished' in str(err):
#                 time.sleep(0.001)
                pass
            else:
                raise err
    
    temp_file_location: str = 'athena_query_results.csv'
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIR}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)

##### enigma_jhud

In [140]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhud LIMIT 46000",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

enigma_jhud = download_and_load_query_results(athena_client, response)
enigma_jhud

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45995,13027.0,Brooks,Georgia,US,2020-04-04T23:34:00,30.839,-83.583,5.0,0.0,0.0,0.0,"""Brooks",csv
45996,36007.0,Broome,New York,US,2020-04-04T23:34:00,42.159,-75.813,65.0,4.0,0.0,0.0,"""Broome",csv
45997,8014.0,Broomfield,Colorado,US,2020-04-04T23:34:00,39.960,-105.057,36.0,2.0,0.0,0.0,"""Broomfield",csv
45998,12011.0,Broward,Florida,US,2020-04-04T23:34:00,26.190,-80.366,1765.0,31.0,0.0,0.0,"""Broward",csv


##### enigma_nytimes_us_county

In [117]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_nytimes_us_county ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

enigma_nytimes_us_county = download_and_load_query_results(athena_client, response)
enigma_nytimes_us_county

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1.0,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1.0,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1.0,0.0
3,2020-01-24,Cook,Illinois,17031.0,1.0,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1.0,0.0
...,...,...,...,...,...,...
26895,2020-04-02,Lincoln,Kentucky,21137.0,1.0,0.0
26896,2020-04-02,Logan,Kentucky,21141.0,2.0,0.0
26897,2020-04-02,Lyon,Kentucky,21143.0,2.0,0.0
26898,2020-04-02,Madison,Kentucky,21151.0,12.0,0.0


##### enigma_nytimes_us_states

In [118]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_nytimes_us_states ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

enigma_nytimes_us_states = download_and_load_query_results(athena_client, response)
enigma_nytimes_us_states

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
3749,2020-05-09,Virginia,51,23196,827
3750,2020-05-09,Washington,53,17763,925
3751,2020-05-09,West Virginia,54,1347,53
3752,2020-05-09,Wisconsin,55,9939,398


##### rearc_covid19_testing_us_total_latest

In [119]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid19_testing_us_total_latest ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

rearc_covid19_testing_us_total_latest = download_and_load_query_results(athena_client, response)
rearc_covid19_testing_us_total_latest

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


##### rearc_covid_19_testing_states_daily

In [120]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_states_daily ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

rearc_covid_19_testing_states_daily = download_and_load_query_results(athena_client, response)
rearc_covid_19_testing_states_daily

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0.0,0.0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0.0,0.0,0.0,0.0,0.0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1.0,0.0,997207b430824ea40b8eb8506c19a93e07bc972e,0.0,0.0,0.0,0.0,0.0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22.0,11.0,50921aeefba3e30d31623aa495b47fb2ecc72fae,0.0,0.0,0.0,0.0,0.0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0.0,0.0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0.0,0.0,0.0,0.0,0.0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5.0,44.0,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,20210119,UT,326221,NaN,1385896.0,NaN,totalTestsViral,2472897,638.0,12645.0,...,NaN,7.0,69.0,c4e353a904e4bb014f1155a257dd31d5bd87d31f,0.0,0.0,0.0,0.0,0.0,NaN
2681,20210119,VA,451076,84917.0,NaN,350.0,totalTestEncountersViral,4878157,3173.0,20066.0,...,NaN,59.0,84.0,1ecd4c4abd7f4e6777e897a255f6d1fb76a7d189,0.0,0.0,0.0,0.0,0.0,NaN
2682,20210119,VI,2260,NaN,37119.0,112.0,posNeg,39379,NaN,NaN,...,NaN,0.0,0.0,12b2c89922709ee808c87895e10d96bf0fdf4d69,0.0,0.0,0.0,0.0,0.0,NaN
2683,20210119,VT,10321,262.0,280334.0,NaN,totalTestsViral,813355,43.0,NaN,...,NaN,0.0,0.0,b42f5e0224e297b2500be4a69fc6d73b9a93c9f7,0.0,0.0,0.0,0.0,0.0,NaN


##### rearc_covid_19_testing_us_daily

In [121]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_us_daily ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

rearc_covid_19_testing_us_daily = download_and_load_query_results(athena_client, response)
rearc_covid_19_testing_us_daily

,date,states,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,...,lastmodified,recovered,total,posneg,deathincrease,hospitalizedincrease,negativeincrease,positiveincrease,totaltestresultsincrease,hash
0,20210307,56,28755524.0,74579770.0,11808.0,40212.0,878613.0,8137.0,45475.0,2801.0,...,2021-03-07T24:00:00Z,NaN,0,0,839,726,130414,41265,1156241,8b26839690cd05c0cef69cb9ed85641a76b5e78e
1,20210306,56,28714259.0,74449356.0,11783.0,41401.0,877887.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1674,503,142201,59620,1409138,d0c0482ea549c9d5c04a7c86acb6fc6a8095a592
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,877384.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,a35ea4289cec4bb55c9f29ae04ec0fd5ac4e0222
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,874603.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,a19ad6379a653834cbda3093791ad2c3b9fab5ff
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,873073.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,9e1d2afda1b0ec243060d6f68a7134d011c0cb2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,20200117,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2020-01-17T24:00:00Z,NaN,0,0,0,0,0,0,0,5fd7ed7fb0f83565ea0aaaf5ff6104e5860be2d6
416,20200116,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2020-01-16T24:00:00Z,NaN,0,0,0,0,0,0,0,4c0f2ea879eefec1a18a93bcd8a382ea56cd452d
417,20200115,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2020-01-15T24:00:00Z,NaN,0,0,0,0,0,0,0,8a449023413eea473277416e9e30ae126c6af047
418,20200114,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2020-01-14T24:00:00Z,NaN,0,0,0,0,0,0,0,fe399466a3dfbe0221a18ed5cacb39a3882d72f8


##### static_datasets_countrycode

In [123]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countrycode ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

static_datasets_countrycode = download_and_load_query_results(athena_client, response)
static_datasets_countrycode

,country,alpha-2 code,alpha-3 code,numeric code,latitude,longitude
0,Afghanistan,AF,AFG,4.0,33.0000,65.0
1,Albania,AL,ALB,8.0,41.0000,20.0
2,Algeria,DZ,DZA,12.0,28.0000,3.0
3,American Samoa,AS,ASM,16.0,-14.3333,-170.0
4,Andorra,AD,AND,20.0,42.5000,1.6
...,...,...,...,...,...,...
251,Wallis and Futuna,WF,WLF,876.0,-13.3000,-176.2
252,Western Sahara,EH,ESH,732.0,24.5000,-13.0
253,Yemen,YE,YEM,887.0,15.0000,48.0
254,Zambia,ZM,ZMB,894.0,-15.0000,30.0


##### static_datasets_countypopulation

In [124]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countypopulation ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

static_datasets_countypopulation = download_and_load_query_results(athena_client, response)
static_datasets_countypopulation

,id,id2,county,state,population estimate 2018
0,0500000US01001,1001,Autauga,Alabama,55601
1,0500000US01003,1003,Baldwin,Alabama,218022
2,0500000US01005,1005,Barbour,Alabama,24881
3,0500000US01007,1007,Bibb,Alabama,22400
4,0500000US01009,1009,Blount,Alabama,57840
...,...,...,...,...,...
3215,0500000US72145,72145,Vega Baja Municipio,Puerto Rico,50185
3216,0500000US72147,72147,Vieques Municipio,Puerto Rico,8364
3217,0500000US72149,72149,Villalba Municipio,Puerto Rico,21476
3218,0500000US72151,72151,Yabucoa Municipio,Puerto Rico,32158


##### rearc_usa_hospital_beds  (json format not correct)

In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_usa_hospital_beds ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

rearc_usa_hospital_beds = download_and_load_query_results(athena_client, response)

##### static_datasets_state_abv

In [129]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_state_abv ",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration = {
        'OutputLocation': S3_STAGING_DIR,
        'EncryptionConfiguration': {'EncryptionOption': 'SSE_S3'},
    },
)
response

static_datasets_state_abv = download_and_load_query_results(athena_client, response)


In [131]:
static_datasets_state_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [132]:
static_datasets_state_abv.columns = static_datasets_state_abv.iloc[0]
static_datasets_state_abv.head()

,State,Abbreviation
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


### Filtering out columns and inner joining them further to create *factCovid*

In [135]:
factCovid1 = enigma_jhud[['fips', 'province_state', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']]

factCovid2 = rearc_covid_19_testing_states_daily[['fips', 'date', 'positive', 'negative', 'hospitalizedcurrently', 'hospitalized', 'hospitalizeddischarged']]

factCovid = pd.merge(factCovid1, factCovid2, on='fips', how='inner') 

In [136]:
factCovid.shape

(8729, 13)

### Filtering out columns and inner joining them further to create *dimRegion*

In [ ]:
dimRegion1 = enigma_jhud[['fips', 'province_state', 'country_region', 'latitude', 'longitude']]
dimRegion2 = enigma_nytimes_us_county[['fips', 'country', 'state']]
dimRegion = pd.merge(dimRegion1, dimRegion2, on='fips', how='inner')

### Filtering out columns and inner joining them further to create *dimDate*

In [144]:
dimDate = rearc_covid_19_testing_states_daily[['fips', 'date']]
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')
dimDate['year'] = dimDate['date'].dt.year
dimDate['month'] = dimDate['date'].dt.month
dimDate['day_of_week'] = dimDate['date'].dt.dayofweek

C:\Users\tmsan\AppData\Local\Temp\ipykernel_10696\112097075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')
C:\Users\tmsan\AppData\Local\Temp\ipykernel_10696\112097075.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year'] = dimDate['date'].dt.year
C:\Users\tmsan\AppData\Local\Temp\ipykernel_10696\112097075.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [145]:
dimDate

,fips,date,year,month,day_of_week
0,2.0,2021-03-07,2021,3,6
1,1.0,2021-03-07,2021,3,6
2,5.0,2021-03-07,2021,3,6
3,60.0,2021-03-07,2021,3,6
4,4.0,2021-03-07,2021,3,6
...,...,...,...,...,...
2680,49.0,2021-01-19,2021,1,1
2681,51.0,2021-01-19,2021,1,1
2682,78.0,2021-01-19,2021,1,1
2683,50.0,2021-01-19,2021,1,1


In [146]:
csv_buffer = StringIO()
csv_buffer

In [151]:
bucket = 'cleanedcovid11'

### Putting *factCovid* to cleaned S3 bucket

In [ ]:
factCovid.to_csv(csv_buffer)  # writing dataframe to the buffer
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/factCovid.csv').put(Body=csv_buffer.getvalue())  # stores the buffer to cleaned S3 bucket

### Putting *dimDate* to cleaned S3 bucket

In [ ]:
dimDate.to_csv(csv_buffer)  # writing dataframe to the buffer
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/dimDate.csv').put(Body=csv_buffer.getvalue())  # stores the buffer to cleaned S3 bucket

### Putting *dimHospital* to cleaned S3 bucket

In [ ]:
dimHospital.to_csv(csv_buffer)  # writing dataframe to the buffer
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/dimHospital.csv').put(Body=csv_buffer.getvalue())  # stores the buffer to cleaned S3 bucket

### Putting *dimRegion* to cleaned S3 bucket

In [ ]:
dimRegion.to_csv(csv_buffer)  # writing dataframe to the buffer
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/dimRegion.csv').put(Body=csv_buffer.getvalue())  # stores the buffer to cleaned S3 bucket